# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 32,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": 100,
                "recon_loss_weight": 10000,
                "ll_weight": 0.001,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": False,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 10,
                "save_step": 10,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                'no_gan': False,
                'monitor_memory': True
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNING - root - Added new config entry: "cfg.model.epochs"
WARNING - root - Added new config entry: "cfg.model.input_repeats"
WARNIN

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-18_13-12__root.log
Starting train_vaegan main
Numpy random seed: 199974232
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 222.00
Initializing VoxelVaegan
['tensorboard', '--logdir', 'current:/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb,best_vaegan:/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb']
Epoch: 0, Elapsed Time: 0.03
Epoch: 0 / 201, Batch: 0 (0 / 32), Elapsed time: 0.03 mins
np.mean(d_real):  0.0
np.mean(d_fake):  0.0
enc_current_lr:  5.527786369235996e-08
dec_current_lr:  5.527786369235996e-08
dis_current_lr:  5.52778636923

Memory Use (GB): 1.6132774353027344
Epoch: 0 / 201, Batch: 18 (0 / 608), Elapsed time: 1.20 mins
np.mean(d_real):  0.20448945
np.mean(d_fake):  0.0044683083
enc_current_lr:  1.1743316665698819e-06
dec_current_lr:  1.1743316665698819e-06
dis_current_lr:  5.9107560238897425e-08
Enc Loss = 56.43, KL Divergence = 0.45, Reconstruction Loss = 0.11, ll_loss = 11508.43, dis_Loss = 6.60, dec_Loss = 80.81, Elapsed time: 1.27 mins
Memory Use (GB): 1.6879768371582031
Epoch: 0 / 201, Batch: 19 (0 / 640), Elapsed time: 1.27 mins
np.mean(d_real):  0.19713995
np.mean(d_fake):  0.0047281394
enc_current_lr:  1.0530420411338893e-06
dec_current_lr:  1.0530420411338893e-06
dis_current_lr:  5.9338242507756914e-08
Enc Loss = 56.45, KL Divergence = 0.43, Reconstruction Loss = 0.11, ll_loss = 13620.50, dis_Loss = 6.82, dec_Loss = 82.92, Elapsed time: 1.34 mins
Memory Use (GB): 1.6895942687988281
Epoch: 0 / 201, Batch: 20 (0 / 672), Elapsed time: 1.34 mins
np.mean(d_real):  0.22131264
np.mean(d_fake):  0.002303

Enc Loss = 57.48, KL Divergence = 0.44, Reconstruction Loss = 0.13, ll_loss = 13395.77, dis_Loss = 6.15, dec_Loss = 82.69, Elapsed time: 2.52 mins
Memory Use (GB): 1.5870704650878906
Epoch: 0 / 201, Batch: 38 (0 / 1248), Elapsed time: 2.52 mins
np.mean(d_real):  0.23115048
np.mean(d_fake):  0.005169314
enc_current_lr:  1.7416918704649596e-06
dec_current_lr:  1.7416918704649596e-06
dis_current_lr:  5.973198712399569e-08
Enc Loss = 51.24, KL Divergence = 0.40, Reconstruction Loss = 0.12, ll_loss = 10990.07, dis_Loss = 6.58, dec_Loss = 80.28, Elapsed time: 2.59 mins
Memory Use (GB): 1.5539703369140625
Epoch: 0 / 201, Batch: 39 (0 / 1280), Elapsed time: 2.59 mins
np.mean(d_real):  0.189167
np.mean(d_fake):  0.0038414476
enc_current_lr:  9.35453830478535e-07
dec_current_lr:  9.35453830478535e-07
dis_current_lr:  5.8554705912724964e-08
Enc Loss = 48.81, KL Divergence = 0.36, Reconstruction Loss = 0.10, ll_loss = 13049.82, dis_Loss = 6.15, dec_Loss = 82.34, Elapsed time: 2.65 mins
Memory Use 

Epoch: 0 / 201, Batch: 57 (0 / 1856), Elapsed time: 3.77 mins
np.mean(d_real):  0.18589193
np.mean(d_fake):  0.0034826365
enc_current_lr:  8.910088310163541e-07
dec_current_lr:  8.910088310163541e-07
dis_current_lr:  5.824058435533245e-08
Enc Loss = 47.85, KL Divergence = 0.37, Reconstruction Loss = 0.12, ll_loss = 10679.69, dis_Loss = 6.50, dec_Loss = 79.96, Elapsed time: 3.83 mins
Memory Use (GB): 1.5852127075195312
Epoch: 0 / 201, Batch: 58 (0 / 1888), Elapsed time: 3.83 mins
np.mean(d_real):  0.17987552
np.mean(d_fake):  0.0027303472
enc_current_lr:  8.147468627243188e-07
dec_current_lr:  8.147468627243188e-07
dis_current_lr:  5.758744832028273e-08
Enc Loss = 48.55, KL Divergence = 0.38, Reconstruction Loss = 0.13, ll_loss = 10732.17, dis_Loss = 6.81, dec_Loss = 80.01, Elapsed time: 3.90 mins
Memory Use (GB): 1.6777153015136719
Epoch: 0 / 201, Batch: 59 (0 / 1920), Elapsed time: 3.90 mins
np.mean(d_real):  0.18602274
np.mean(d_fake):  0.002997704
enc_current_lr:  8.927433140383175e

...

Enc Loss = 39.00, KL Divergence = 0.28, Reconstruction Loss = 0.13, ll_loss = 10702.29, dis_Loss = 6.68, dec_Loss = 79.92, Elapsed time: 11.47 mins
Memory Use (GB): 1.55511474609375
Epoch: 1 / 201, Batch: 2 (96 / 5504), Elapsed time: 11.47 mins
np.mean(d_real):  0.19394118
np.mean(d_fake):  0.0036384417
enc_current_lr:  1.0042038367361906e-06
dec_current_lr:  1.0042038367361906e-06
dis_current_lr:  5.8376776721029764e-08
Enc Loss = 35.14, KL Divergence = 0.26, Reconstruction Loss = 0.12, ll_loss = 9061.79, dis_Loss = 7.05, dec_Loss = 78.28, Elapsed time: 11.53 mins
Memory Use (GB): 1.5584068298339844
Epoch: 1 / 201, Batch: 3 (128 / 5504), Elapsed time: 11.53 mins
np.mean(d_real):  0.14389864
np.mean(d_fake):  0.0029577469
enc_current_lr:  4.7657630704723113e-07
dec_current_lr:  4.7657630704723113e-07
dis_current_lr:  5.778410048560881e-08
Enc Loss = 36.39, KL Divergence = 0.25, Reconstruction Loss = 0.11, ll_loss = 11298.96, dis_Loss = 6.24, dec_Loss = 80.52, Elapsed time: 11.60 mins
M

Memory Use (GB): 1.7086524963378906
Epoch: 1 / 201, Batch: 21 (704 / 5504), Elapsed time: 12.72 mins
np.mean(d_real):  0.18285891
np.mean(d_fake):  0.0027402258
enc_current_lr:  8.517180273972178e-07
dec_current_lr:  8.517180273972178e-07
dis_current_lr:  5.7595977234810276e-08
Enc Loss = 38.01, KL Divergence = 0.26, Reconstruction Loss = 0.11, ll_loss = 11890.06, dis_Loss = 6.20, dec_Loss = 81.10, Elapsed time: 12.78 mins
Memory Use (GB): 1.7258987426757812
Epoch: 1 / 201, Batch: 22 (736 / 5504), Elapsed time: 12.78 mins
np.mean(d_real):  0.20121697
np.mean(d_fake):  0.0027556578
enc_current_lr:  1.118708111361416e-06
dec_current_lr:  1.118708111361416e-06
dis_current_lr:  5.760930342628576e-08
Enc Loss = 37.10, KL Divergence = 0.25, Reconstruction Loss = 0.11, ll_loss = 12241.67, dis_Loss = 5.81, dec_Loss = 81.45, Elapsed time: 12.85 mins
Memory Use (GB): 1.6696281433105469
Epoch: 1 / 201, Batch: 23 (768 / 5504), Elapsed time: 12.85 mins
np.mean(d_real):  0.22727373
np.mean(d_fake): 

np.mean(d_real):  0.21899621
np.mean(d_fake):  0.0042049973
enc_current_lr:  1.4556461881910834e-06
dec_current_lr:  1.4556461881910834e-06
dis_current_lr:  5.8874702807619625e-08
Enc Loss = 34.93, KL Divergence = 0.24, Reconstruction Loss = 0.11, ll_loss = 11381.18, dis_Loss = 6.48, dec_Loss = 80.58, Elapsed time: 14.03 mins
Memory Use (GB): 1.6240806579589844
Epoch: 1 / 201, Batch: 41 (1344 / 5504), Elapsed time: 14.03 mins
np.mean(d_real):  0.20824733
np.mean(d_fake):  0.003317289
enc_current_lr:  1.2415823424366044e-06
dec_current_lr:  1.2415823424366044e-06
dis_current_lr:  5.8096398151127255e-08
Enc Loss = 37.72, KL Divergence = 0.25, Reconstruction Loss = 0.12, ll_loss = 12610.13, dis_Loss = 6.36, dec_Loss = 81.81, Elapsed time: 14.09 mins
Memory Use (GB): 1.6574249267578125
Epoch: 1 / 201, Batch: 42 (1376 / 5504), Elapsed time: 14.09 mins
np.mean(d_real):  0.21735862
np.mean(d_fake):  0.0029870532
enc_current_lr:  1.420827280524533e-06
dec_current_lr:  1.420827280524533e-06
dis

Enc Loss = 32.58, KL Divergence = 0.21, Reconstruction Loss = 0.09, ll_loss = 11882.87, dis_Loss = 7.47, dec_Loss = 81.07, Elapsed time: 15.28 mins
Memory Use (GB): 1.7670516967773438
Epoch: 1 / 201, Batch: 60 (1952 / 5504), Elapsed time: 15.28 mins
np.mean(d_real):  0.17256558
np.mean(d_fake):  0.0022895974
enc_current_lr:  7.307524820866136e-07
dec_current_lr:  7.307524820866136e-07
dis_current_lr:  5.7208196322187135e-08
Enc Loss = 34.53, KL Divergence = 0.22, Reconstruction Loss = 0.10, ll_loss = 12561.95, dis_Loss = 7.10, dec_Loss = 81.75, Elapsed time: 15.34 mins
Memory Use (GB): 1.7471885681152344
Epoch: 1 / 201, Batch: 61 (1984 / 5504), Elapsed time: 15.34 mins
np.mean(d_real):  0.19971038
np.mean(d_fake):  0.0044663223
enc_current_lr:  1.0939837266867795e-06
dec_current_lr:  1.0939837266867795e-06
dis_current_lr:  5.9105800451414095e-08
Enc Loss = 35.98, KL Divergence = 0.25, Reconstruction Loss = 0.12, ll_loss = 11142.28, dis_Loss = 6.17, dec_Loss = 80.33, Elapsed time: 15.41

Memory Use (GB): 1.69378662109375
Epoch: 1 / 201, Batch: 79 (2560 / 5504), Elapsed time: 16.53 mins
np.mean(d_real):  0.19772795
np.mean(d_fake):  0.0041379975
enc_current_lr:  1.0622718507365647e-06
dec_current_lr:  1.0622718507365647e-06
dis_current_lr:  5.881559840301105e-08
Enc Loss = 33.34, KL Divergence = 0.21, Reconstruction Loss = 0.10, ll_loss = 12453.11, dis_Loss = 6.68, dec_Loss = 81.63, Elapsed time: 16.59 mins
Memory Use (GB): 1.6025657653808594
Epoch: 1 / 201, Batch: 80 (2592 / 5504), Elapsed time: 16.60 mins
np.mean(d_real):  0.21951157
np.mean(d_fake):  0.003116678
enc_current_lr:  1.4667767574038765e-06
dec_current_lr:  1.4667767574038765e-06
dis_current_lr:  5.79219403171848e-08
Enc Loss = 31.23, KL Divergence = 0.23, Reconstruction Loss = 0.11, ll_loss = 8673.02, dis_Loss = 5.60, dec_Loss = 77.85, Elapsed time: 16.66 mins
Memory Use (GB): 1.655548095703125
Epoch: 1 / 201, Batch: 81 (2624 / 5504), Elapsed time: 16.66 mins
np.mean(d_real):  0.16683084
np.mean(d_fake): 